In [1]:
import torch
from torch import optim
from torch.optim.adam import Adam

import utils

In [4]:

cfgs = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

pretrained_vgg = utils.VGG(cfgs['VGG16'])
pretrained_vgg.load_state_dict(torch.load('vgg16-89.33%.pth')) # type: ignore


bayesian_vgg = utils.BayesianVGG(cfgs['VGG16'])
bayesian_vgg.from_vgg(pretrained_vgg)


optimizer = Adam(bayesian_vgg.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.6)


def reg_coef_lambda(epoch: int) -> float:
    return min(1., 0.5 ** (8 - epoch // 50))


train_loader, test_loader = utils.get_cifar_dataloaders()


trainer = utils.BayesianModelTrainer(bayesian_vgg, train_loader, test_loader, optimizer, scheduler, epochs=500, reg_coef_lambda=reg_coef_lambda)


In [8]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin


accuracy,▁▃▆▆▇▇███▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_loss,▁▄▆▆▇▇▆▇██▇███▇▇▆▇▆▆▇▆▅▅▆▆▆▆▆▆▆▆▇▆▇▆▆▆▇▆
loss,▄▁▁▁▂▃▁▁▁▂▂▁▂▂▁▁▁▂▂▁▅▃▄▂▄▂▂▅▅▄▅▅▅█▄▅▆▄▅▆
neuron_sparsity_0,▁▁▁▁▁▁▁▁▁▁▁▂▄▄▄▇▇▇▇▇████████████████████
neuron_sparsity_1,▅▂▂▂▁▃▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████
neuron_sparsity_10,▁▃▆▃▅▃▄▅▆▇██████████████████████████████
neuron_sparsity_11,▁▄▇▄▇▇▇▇████████████████████████████████
neuron_sparsity_12,▁▁▂▁▃▄▄▄▄▅▅▅▆▆▇▇▇▇██████████████████████
neuron_sparsity_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
neuron_sparsity_3,▁▂▂▁▅▁▇█████████████████████████████████


In [13]:
torch.save(bayesian_vgg.state_dict(), 'bayesian-vgg16.pth') # type: ignore